In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
%tensorflow_version 1.x 
import tensorflow as tf
import keras
from keras import backend as K
from keras import models
from keras import layers 
from keras import optimizers
from keras.models import model_from_json
import numpy as np
import pandas as pd

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
pip install flask_ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,request

In [ ]:
%mkdir templates -p #create templates directory 
%mkdir static -p    #create static directory

In [ ]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/megaproject2/flask/state_district.js static/state_district.js   #copy javascript file to static directory
!cp /gdrive/My\ Drive/Colab\ Notebooks/megaproject2/flask/output.html templates/output.html            #copy html files to templates directory
!cp /gdrive/My\ Drive/Colab\ Notebooks/megaproject2/flask/yield_input.html templates/yield_input.html

In [ ]:
def prod(input_district,input_year,input_season,input_crop,input_area):
  train=pd.read_csv('/gdrive/My Drive/Colab Notebooks/megaproject2/train.csv')
  district_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/megaproject2/Classified_data/district.csv')
  season_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/megaproject2/Classified_data/season.csv')
  crop_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/megaproject2/Classified_data/crop.csv')

  '''normalization for train'''
  cols_to_norm = ['Crop_Year']
  meanyear = train[cols_to_norm].mean()
  train[cols_to_norm] -= meanyear
  stdyear = train[cols_to_norm].std()
  train[cols_to_norm] /=stdyear


  cols_to_norm = ['Area']
  meanarea = train[cols_to_norm].mean()
  train[cols_to_norm] -= meanarea
  stdarea = train[cols_to_norm].std()
  train[cols_to_norm] /=stdarea

  cols_to_norm = ['Production']
  meanprod = train[cols_to_norm].mean()
  train[cols_to_norm] -= meanprod
  stdprod = train[cols_to_norm].std()
  train[cols_to_norm] /=stdprod

  input_year -=meanyear
  input_year /=stdyear
  input_area -=meanarea
  input_area /=stdarea

  # to convert string to integer 
  district1=district_data.loc[district_data['0']==input_district].index
  season1=season_data.loc[season_data['0']==input_season].index
  crop1=crop_data.loc[crop_data['0']==input_crop].index
  year1=input_year
  area1=input_area

  district1=np.asarray(district1,dtype='float32')
  year1=np.asarray(year1,dtype='float32')
  season1=np.asarray(season1,dtype='float32')
  crop1=np.asarray(crop1,dtype='float32')
  area1=np.asarray(area1,dtype='float32')
  
  K.clear_session()
  # load json and create model
  json_file = open('/gdrive/My Drive/Colab Notebooks/megaproject2/Saved_model/embedded_model4.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights("/gdrive/My Drive/Colab Notebooks/megaproject2/Saved_model/embedded_model4.h5")

  loaded_model.compile(optimizer='RMSprop', loss='mse', metrics=['mape'])
  production=loaded_model.predict([district1,year1,season1,crop1,area1])
  K.clear_session()
  production*=stdprod
  production+=meanprod
  if production < 0:
    production=0
  production=production[0][0]  
  return(production)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/')
def home():
   return render_template('yield_input.html')

@app.route('/output',methods = ['POST', 'GET'])
def output():
   if request.method == 'POST':
      state = request.form['state']
      district=request.form['district']
      year=request.form['year']
      season=request.form['season']
      crop=request.form['crop']
      area=request.form['area']
      year=int(year)
      area=float(area)
      production=prod(district,year,season,crop,area)
      return render_template('output.html',state=state,district=district,year=year,season=season,crop=crop,area=area,production=production)

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2beb59fb116b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Sep/2020 15:25:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2020 15:25:50] "GET /css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2020 15:25:51] "GET /jquery-3.3.1.js HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2020 15:25:52] "GET /static/state_district.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2020 15:26:17] "GET /favicon.ico HTTP/1.1" 404 -


Instructions for updating:
If using Keras pass *_constraint arguments to layers.




127.0.0.1 - - [18/Sep/2020 15:27:05] "POST /output HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2020 15:27:06] "GET /css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2020 15:27:06] "GET /jquery-3.3.1.js HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2020 15:29:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2020 15:29:18] "GET /css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2020 15:29:18] "GET /jquery-3.3.1.js HTTP/1.1" 404 -
